In [123]:
# Read in training or test data file
#
def read_data(fname):
    exemplars = []
    file = open(fname, 'r');
    for line in file:
        data = tuple([w.lower() for w in line.split()])
        exemplars += [ (data[0::2], data[1::2]), ]

    return exemplars 

In [175]:
trainData = read_data("bc.train")
testData= read_data("bc.test") 

In [144]:
Ytrain = train(trainData)

In [165]:
#Making Trigram Model

AA= {} 
for tag_seq in Ytrain: 
    for i in range(len(tag_seq)-2):
        if (tag_seq[i],tag_seq[i+1]) not in AA:
            AA[(tag_seq[i],tag_seq[i+1])] = {}
            AA[(tag_seq[i],tag_seq[i+1])][tag_seq[i+2]] = 1
        elif tag_seq[i+2] not in  AA[(tag_seq[i],tag_seq[i+1])]:
            AA[(tag_seq[i],tag_seq[i+1])][tag_seq[i+2]] = 1 
        else:
            AA[(tag_seq[i],tag_seq[i+1])][tag_seq[i+2]] += 1
## Changing in probability matrix
for states in AA:
    sum =0
    for state in AA[states]:
        sum += AA[states][state]
    for state in AA[states]:
        AA[states][state]= AA[states][state]*1.0/sum
    

In [174]:
AA[(7,3)]

{0: 0.18289033620841402,
 1: 0.12902657982749516,
 2: 0.05139940151381799,
 3: 0.09276535821158247,
 4: 0.18676289385671538,
 5: 0.08044358387607815,
 6: 0.09434958634043303,
 7: 0.015138179897905299,
 8: 0.06724168280232354,
 9: 0.08889280056328111,
 10: 0.010737546206653759,
 11: 0.0003520506953001232}

In [182]:
def train(data):
    word2idx,tag2idx = {},{}  #Word Vocabulary with word as key and word_idx as value ,#Tag Vocabulary with Tag as key and Tag_idx as value
    word_idx,tag_idx = 0,0  #counter for words , tags
    Xtrain,Ytrain = [],[]    #for storing overall Sentence & Tag sequences
    currentX,currentY = [],[]  #to store current word sequences (observed Data) & tag sequences (unobserved Data)
    for sentence in data:
        for word,tag in zip(sentence[0],sentence[1]): 
            if word not in word2idx:
                    word2idx[word] = word_idx
                    word_idx += 1
            currentX.append(word2idx[word])

            if tag not in tag2idx: 
                tag2idx[tag] = tag_idx
                tag_idx += 1
            currentY.append(tag2idx[tag])
        Xtrain.append(currentX)
        Ytrain.append(currentY)
        currentX = []
        currentY = []

    V = len(word2idx) + 1  #Vocabulary Size

    # find hidden state, transition matrix and pi
    M = max(max(y) for y in Ytrain) + 1 #find total number of hiddent states
    A = np.ones((M, M))*(10e-2) # add-one smoothing
    pi = np.zeros(M) #initial Matrix
    for y in Ytrain:
        pi[y[0]] += 1
        for i in xrange(len(y)-1):
            A[y[i], y[i+1]] += 1

    return A,tag2idx

In [183]:
A,tag2idx=train(trainData)

In [181]:
A.sum(axis=1,keepdims=True)

array([[ 114262.2],
       [ 235891.2],
       [  72697.2],
       [ 146148.2],
       [ 123936.2],
       [  71523.2],
       [  44594.2],
       [  31503.2],
       [  22381.2],
       [  33974.2],
       [  13495.2],
       [   1201.2]])

In [184]:
tag2idx

{'.': 5,
 'adj': 2,
 'adp': 4,
 'adv': 6,
 'conj': 7,
 'det': 0,
 'noun': 1,
 'num': 10,
 'pron': 9,
 'prt': 8,
 'verb': 3,
 'x': 11}

In [ ]:
def HMM(self, x):
    # returns the most likely state sequence given observed sequence x
    # using the Viterbi algorithm 

    x = [self.word2idx[word] for word in x] 

    T = len(x)
    delta = np.zeros((T, self.M))
    psi = np.zeros((T, self.M))
    delta[0] = np.log(self.pi) + np.log(self.B[:,x[0]]) 
    for t in xrange(1, T-1):
        for j in xrange(self.M): 
            for k in xrange(self.M)
                if t==1:
                    delta[t,j] = np.max(delta[t-1] + np.log(self.A[:,j])) + np.log(self.B[j, x[t]])
                    psi[t,j] = np.argmax(delta[t-1] + np.log(self.A[:,j]))
                else: 
                    delta[t,j] = np.max(delta[t-1]+ np.log(self.A[j:,k) + np.log(self.B[j, x[t]])
                    psi[t,j] = np.argmax(delta[t-1] + np.log(self.A[:,j]))

    # backtrack
    states = np.zeros(T, dtype=np.int32)
    states[T-1] = np.argmax(delta[T-1])
    for t in xrange(T-2, -1, -1):
        states[t] = psi[t+1, states[t+1]]

    tag_states=[]
    for i in range(len(states)):
        for tag,tag_idx in self.tag2idx.iteritems():
            if tag_idx==states[i]: tag_states.append(tag)

    return [[tag_states],[]]


In [98]:


class Solver:
    def __init__(self): 
        self.M =None
        self.pi=None #initial
        self.A =None #Transition
        self.B =None #emission
        self.word2idx =None
        self.tag2idx= None  
        

            
    def train(self,data):
        word2idx,tag2idx = {},{}  #Word Vocabulary with word as key and word_idx as value ,#Tag Vocabulary with Tag as key and Tag_idx as value
        word_idx,tag_idx = 0,0  #counter for words , tags
        Xtrain,Ytrain = [],[]    #for storing overall Sentence & Tag sequences
        currentX,currentY = [],[]  #to store current word sequences (observed Data) & tag sequences (unobserved Data)
        for sentence in data:
            for word,tag in zip(sentence[0],sentence[1]): 
                if word not in word2idx:
                        word2idx[word] = word_idx
                        word_idx += 1
                currentX.append(word2idx[word])

                if tag not in tag2idx: 
                    tag2idx[tag] = tag_idx
                    tag_idx += 1
                currentY.append(tag2idx[tag])
            Xtrain.append(currentX)
            Ytrain.append(currentY)
            currentX = []
            currentY = []

        V = len(word2idx) + 1  #Vocabulary Size

        # find hidden state, transition matrix and pi
        M = max(max(y) for y in Ytrain) + 1 #find total number of hiddent states
        A = np.ones((M, M))*(10e-2) # add-one smoothing
        pi = np.zeros(M) #initial Matrix

        for y in Ytrain:
            pi[y[0]] += 1
            for i in xrange(len(y)-1):
                A[y[i], y[i+1]] += 1

        # turn it into a probability matrix
        A /= A.sum(axis=1, keepdims=True) #state transition matrix
        pi /= pi.sum() #emission matrix

        # find the observation matrix
        B = np.ones((M, V))*(10e-2) # add smoothing
        for x, y in zip(Xtrain, Ytrain):
            for xi, yi in zip(x, y):
                B[yi, xi] += 1
        B /= B.sum(axis=1, keepdims=True)

        self.M=M
        self.pi = pi
        self.A = A
        self.B = B 
        self.word2idx = word2idx
        self.tag2idx= tag2idx  

    def HMM(self, x):
        # returns the most likely state sequence given observed sequence x
        # using the Viterbi algorithm 
        
        x = [self.word2idx[word] for word in x] 

        T = len(x)
        delta = np.zeros((T, self.M))
        psi = np.zeros((T, self.M))
        delta[0] = np.log(self.pi) + np.log(self.B[:,x[0]]) 
        for t in xrange(1, T):
            for j in xrange(self.M):
                delta[t,j] = np.max(delta[t-1] + np.log(self.A[:,j])) + np.log(self.B[j, x[t]])
                psi[t,j] = np.argmax(delta[t-1] + np.log(self.A[:,j]))

        # backtrack
        states = np.zeros(T, dtype=np.int32)
        states[T-1] = np.argmax(delta[T-1]) #Last word most probable state
        for t in xrange(T-2, -1, -1):
            states[t] = psi[t+1, states[t+1]]
        
        tag_states=[]
        for i in range(len(states)):
            for tag,tag_idx in self.tag2idx.iteritems():
                if tag_idx==states[i]: tag_states.append(tag)
            
        return [[tag_states],[]]

    

In [100]:
solve = Solver()
solve.train(trainData)
states=solve.HMM(s)

In [101]:
picking

[[['adp',
   'det',
   'adj',
   'noun',
   '.',
   'noun',
   'verb',
   'det',
   'noun',
   'conj',
   'verb',
   'pron',
   'adp',
   'det',
   'adj',
   'noun',
   '.']],
 []]

In [104]:
solve.B.shape

(12, 45298)

In [117]:
x=np.array(([2,8,3],[4,5,6]),dtype=int)

In [119]:
 np.argmax(x[:,2])

1

In [109]:
np.max(x)

3

In [110]:
np.argmax(x)

2

In [120]:
x=[1,2,3,4]

In [121]:
x[:-1]

[1, 2, 3]

In [122]:
float("{0:.2f}".format(0.000000008))

0.0